<a href="https://colab.research.google.com/github/MAlam1802/begining-bioinformatics/blob/main/Cross_Ancestry_PRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Core scientific + plotting stack
!pip install -q \
    pandas \
    numpy \
    scipy \
    scikit-learn \
    statsmodels \
    matplotlib \
    seaborn \
    tqdm \
    pyarrow


In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting defaults
sns.set(style="whitegrid", context="notebook")
plt.rcParams["figure.figsize"] = (7, 5)

print("Libraries imported and ready.")


Libraries imported and ready.


In [3]:
# install plink 1.9 binary
!wget -qO- http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20201019.zip > plink.zip
!unzip -o plink.zip
!chmod +x plink
!mv plink /usr/local/bin/

!plink --version


Archive:  plink.zip
  inflating: plink                   
  inflating: LICENSE                 
  inflating: toy.ped                 
  inflating: toy.map                 
  inflating: prettify                
PLINK v1.90b6.21 64-bit (19 Oct 2020)


In [4]:
import pandas as pd

eur_path = "GCST90320058EUR.h.tsv.gz"
afr_path = "GCST90320061AFR.h.tsv.gz"

# Loading with pandas
eur_df = pd.read_csv(eur_path, sep="\t", compression="gzip", low_memory=False)
afr_df = pd.read_csv(afr_path, sep="\t", compression="gzip", low_memory=False)

print("Loaded EUR GWAS:", eur_df.shape)
print("Loaded AFR GWAS:", afr_df.shape)
print("EUR columns:", list(eur_df.columns))
print("AFR columns:", list(afr_df.columns))


Loaded EUR GWAS: (13264860, 17)
Loaded AFR GWAS: (19077597, 17)
EUR columns: ['chromosome', 'base_pair_location', 'effect_allele', 'other_allele', 'beta', 'standard_error', 'effect_allele_frequency', 'p_value', 'variant_id', 'rsid', 'het_i2', 'het_p_value', 'n_samples', 'n_cases', 'n_studies', 'hm_coordinate_conversion', 'hm_code']
AFR columns: ['chromosome', 'base_pair_location', 'effect_allele', 'other_allele', 'beta', 'standard_error', 'effect_allele_frequency', 'p_value', 'variant_id', 'rsid', 'het_i2', 'het_p_value', 'n_samples', 'n_cases', 'n_studies', 'hm_coordinate_conversion', 'hm_code']


In [5]:
import numpy as np

def clean_gwas(df, label="GWAS"):
    df = df.copy()
    print(f"\n=== Cleaning {label} ===")
    print("Original columns:", list(df.columns))

    # --- 1. Chromosome and position ---
    df["CHR"] = pd.to_numeric(df["chromosome"], errors="coerce")
    df["BP"]  = pd.to_numeric(df["base_pair_location"], errors="coerce")

    # --- 2. SNP ID (rsid preferred) ---
    if "rsid" in df.columns:
        df["SNP"] = df["rsid"]
    else:
        df["SNP"] = df["variant_id"]

    before = df.shape[0]
    df = df[~df["SNP"].isna()]
    print(f"{label}: Dropped {before - df.shape[0]} rows with missing SNP IDs.")

    # --- 3. Alleles ---
    df["A1"] = df["effect_allele"].astype(str).str.upper()
    df["A2"] = df["other_allele"].astype(str).str.upper()

    # --- 4. Effect size, SE, p-value ---
    df["BETA"] = pd.to_numeric(df["beta"], errors="coerce")
    df["SE"]   = pd.to_numeric(df["standard_error"], errors="coerce")
    df["P"]    = pd.to_numeric(df["p_value"], errors="coerce")

    # --- 5. Sample size (n_samples) ---
    df["N"] = pd.to_numeric(df["n_samples"], errors="coerce")

    # --- 6. Keep autosomes only (1–22) ---
    before = df.shape[0]
    df = df[df["CHR"].between(1, 22, inclusive="both")]
    print(f"{label}: Dropped {before - df.shape[0]} non-autosomal rows.")

    # --- 7. Drop rows with missing critical fields ---
    before = df.shape[0]
    df = df.dropna(subset=["CHR", "BP", "SNP", "A1", "A2", "BETA", "SE", "P"])
    print(f"{label}: Dropped {before - df.shape[0]} rows with missing fields.")

    # --- 8. Ensure numeric sanity ---
    before = df.shape[0]
    df = df[
        np.isfinite(df["BETA"]) &
        np.isfinite(df["SE"]) &
        np.isfinite(df["P"]) &
        (df["SE"] > 0)
    ]
    print(f"{label}: Dropped {before - df.shape[0]} rows with invalid numeric values.")

    # --- 9. Final clean selection ---
    clean = df[["CHR", "BP", "SNP", "A1", "A2", "BETA", "SE", "P", "N"]].copy()
    clean = clean.sort_values(["CHR", "BP"]).reset_index(drop=True)

    print(f"{label}: Final clean shape: {clean.shape}")
    return clean

# Clean both GWAS tables
eur_clean = clean_gwas(eur_df, "EUR_ccRCC")
afr_clean = clean_gwas(afr_df, "AFR_ccRCC")

# Save full clean versions
eur_clean_path = "EUR_ccRCC.clean.autosomes.tsv"
afr_clean_path = "AFR_ccRCC.clean.autosomes.tsv"

eur_clean.to_csv(eur_clean_path, sep="\t", index=False)
afr_clean.to_csv(afr_clean_path, sep="\t", index=False)

print("\nSaved clean files:")
print(" ", eur_clean_path)
print(" ", afr_clean_path)

# Create minimal clumping input files
eur_for_clump = eur_clean[["SNP", "CHR", "BP", "P", "A1", "A2", "BETA"]]
afr_for_clump = afr_clean[["SNP", "CHR", "BP", "P", "A1", "A2", "BETA"]]

eur_for_clump_path = "EUR_ccRCC.for_clump.tsv"
afr_for_clump_path = "AFR_ccRCC.for_clump.tsv"

eur_for_clump.to_csv(eur_for_clump_path, sep="\t", index=False)
afr_for_clump.to_csv(afr_for_clump_path, sep="\t", index=False)

print("\nSaved clump files:")
print(" ", eur_for_clump_path)
print(" ", afr_for_clump_path)



=== Cleaning EUR_ccRCC ===
Original columns: ['chromosome', 'base_pair_location', 'effect_allele', 'other_allele', 'beta', 'standard_error', 'effect_allele_frequency', 'p_value', 'variant_id', 'rsid', 'het_i2', 'het_p_value', 'n_samples', 'n_cases', 'n_studies', 'hm_coordinate_conversion', 'hm_code']
EUR_ccRCC: Dropped 0 rows with missing SNP IDs.
EUR_ccRCC: Dropped 445595 non-autosomal rows.
EUR_ccRCC: Dropped 0 rows with missing fields.
EUR_ccRCC: Dropped 0 rows with invalid numeric values.
EUR_ccRCC: Final clean shape: (12819265, 9)

=== Cleaning AFR_ccRCC ===
Original columns: ['chromosome', 'base_pair_location', 'effect_allele', 'other_allele', 'beta', 'standard_error', 'effect_allele_frequency', 'p_value', 'variant_id', 'rsid', 'het_i2', 'het_p_value', 'n_samples', 'n_cases', 'n_studies', 'hm_coordinate_conversion', 'hm_code']
AFR_ccRCC: Dropped 0 rows with missing SNP IDs.
AFR_ccRCC: Dropped 698435 non-autosomal rows.
AFR_ccRCC: Dropped 0 rows with missing fields.
AFR_ccRCC: Dr

In [6]:
# Verifying columns
import pandas as pd
import numpy as np

eur_clean = pd.read_csv("EUR_ccRCC.clean.autosomes.tsv", sep="\t")
afr_clean = pd.read_csv("AFR_ccRCC.clean.autosomes.tsv", sep="\t")

eur_clump = pd.read_csv("EUR_ccRCC.for_clump.tsv", sep="\t")
afr_clump = pd.read_csv("AFR_ccRCC.for_clump.tsv", sep="\t")

print("EUR clean shape:", eur_clean.shape)
print("AFR clean shape:", afr_clean.shape)
print("EUR clump shape:", eur_clump.shape)
print("AFR clump shape:", afr_clump.shape)

print("\nEUR clean columns:", eur_clean.columns.tolist())
print("AFR clean columns:", afr_clean.columns.tolist())
print("\nEUR clump columns:", eur_clump.columns.tolist())
print("AFR clump columns:", afr_clump.columns.tolist())


EUR clean shape: (12819265, 9)
AFR clean shape: (18379162, 9)
EUR clump shape: (12819265, 7)
AFR clump shape: (18379162, 7)

EUR clean columns: ['CHR', 'BP', 'SNP', 'A1', 'A2', 'BETA', 'SE', 'P', 'N']
AFR clean columns: ['CHR', 'BP', 'SNP', 'A1', 'A2', 'BETA', 'SE', 'P', 'N']

EUR clump columns: ['SNP', 'CHR', 'BP', 'P', 'A1', 'A2', 'BETA']
AFR clump columns: ['SNP', 'CHR', 'BP', 'P', 'A1', 'A2', 'BETA']


In [7]:
# Checking consistency and missingness
def quick_qc(df, label):
    print(f"\n=== {label} ===")
    print(df.head())
    print("\nDtypes:")
    print(df.dtypes)
    print("\nMissing values per column:")
    print(df.isna().sum())

quick_qc(eur_clean, "EUR_clean")
quick_qc(afr_clean, "AFR_clean")



=== EUR_clean ===
   CHR     BP           SNP A1 A2      BETA        SE         P       N
0    1  13668     rs2691328  A  G -0.482482  0.570721  0.397893  305873
1    1  14773   rs878915777  T  C  0.059339  0.329717  0.857176  305873
2    1  14860   rs533499096  T  G -0.283933  0.318174  0.372188  305873
3    1  17626  rs1230374648  A  G -0.017238  0.402357  0.965828  305873
4    1  17765   rs373918278  A  G  0.562539  0.497359  0.258033  305873

Dtypes:
CHR       int64
BP        int64
SNP      object
A1       object
A2       object
BETA    float64
SE      float64
P       float64
N         int64
dtype: object

Missing values per column:
CHR     0
BP      0
SNP     0
A1      0
A2      0
BETA    0
SE      0
P       0
N       0
dtype: int64

=== AFR_clean ===
   CHR      BP          SNP A1 A2      BETA        SE         P     N
0    1  709040  rs144745434  C  T -0.156402  0.340545  0.646041  3526
1    1  722777  rs879352363  A  G -0.630476  0.479140  0.188225  3024
2    1  722858  rs8798

In [8]:
print("EUR variants:", eur_clean.shape[0])
print("AFR variants:", afr_clean.shape[0])


EUR variants: 12819265
AFR variants: 18379162
